# User Interface

> Gradio interface for the chat application.

In [ ]:
#| default_exp ui

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
import gradio as gr
from typing import List, Tuple, Generator, Dict, Any, Optional
from gradiochat.config import ChatAppConfig, ModelConfig
from gradiochat.app import BaseChatApp
from pathlib import Path

In [ ]:
#| export
class GradioChat:
    """Gradio interface for the chat application"""
    
    def __init__(self, app: BaseChatApp):
        """Initialize with a configured BaseChatApp"""
        self.app = app
        self.interface = None
    
    def respond(self, message: str, chat_history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
        """Generate a response to the user message and update chat history"""
        # Store the current chat history in the app
        self.app.chat_history = chat_history
        
        # Generate response
        response = self.app.generate_response(message)
        
        # Update chat history
        chat_history.append((message, response))
        
        # Return empty message (to clear input) and updated history
        return "", chat_history
    
    def respond_stream(self, message: str, chat_history: List[Tuple[str, str]]) -> Generator[Tuple[str, List[Tuple[str, str]]], None, None]:
        """Generate a streaming response to the user message"""
        # Store the current chat history in the app
        self.app.chat_history = chat_history
        
        # Add user message to history with empty assistant response
        chat_history.append((message, ""))
        
        # Stream the response
        accumulated_text = ""
        for text_chunk in self.app.generate_stream(message):
            accumulated_text += text_chunk
            
            # Update the last assistant message
            updated_history = chat_history[:-1] + [(message, accumulated_text)]
            
            # Yield empty message and updated history
            yield "", updated_history
    
    def build_interface(self) -> gr.Blocks:
        """Build and return the Gradio interface"""
        with gr.Blocks(theme=self.app.config.theme) as interface:
            with gr.Row():
                # Left column for logo
                with gr.Column(scale=1):
                    if self.app.config.logo_path:
                        gr.Image(value=self.app.config.logo_path,
                            show_label=False,
                            container=False,
                            show_download_button=False,
                            show_fullscreen_button=False,
                            height=80,
                            width=80)
                    else:
                        gr.Image(value=None,
                            show_label=False,
                            container=False,
                            show_download_button=False,
                            show_fullscreen_button=False,
                            height=80,
                            width=80)
                with gr.Column(scale=4):
                    # App title and description
                    gr.Markdown(f"# {self.app.config.app_name}")
                    if self.app.config.description:
                        gr.Markdown(self.app.config.description)
            
            # Chat interface
            chatbot = gr.Chatbot(height=500, label="Conversation")
            msg = gr.Textbox(
                placeholder="Type your message here...",
                label="Your message",
                lines=2
            )
            
            # Buttons
            with gr.Row():
                submit_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.ClearButton([msg, chatbot], value="Clear chat")
            
            # System prompt and context viewer (collapsible)
            with gr.Accordion("View System Information", open=False):
                if self.app.config.show_system_prompt:
                    gr.Markdown(f"### System Prompt\n{self.app.config.system_prompt}")
                
                if self.app.config.show_context and hasattr(self.app, 'context_text') and self.app.context_text:
                    gr.Markdown(f"### Additional Context\n{self.app.context_text}")
            
            # Set up event handlers
            submit_btn.click(
                self.respond,
                inputs=[msg, chatbot],
                outputs=[msg, chatbot]
            )
            
            msg.submit(
                self.respond,
                inputs=[msg, chatbot],
                outputs=[msg, chatbot]
            )
            
            # Initialize with starter prompt if available
            if self.app.config.starter_prompt:
                chatbot.value = [("", self.app.config.starter_prompt)]
            
            self.interface = interface
            return interface
    
    def launch(self, **kwargs):
        """Launch the Gradio interface"""
        if self.interface is None:
            self.build_interface()
        
        return self.interface.launch(**kwargs)

In [ ]:
#| export
def create_chat_app(config: ChatAppConfig) -> GradioChat:
    """Create a complete chat application from a configuration"""
    base_app = BaseChatApp(config)
    return GradioChat(base_app)

In [ ]:
# Test cell - not for export
themeWDODelta = gr.themes.Base(
    primary_hue=gr.themes.Color(c100="#ffedd5", c200="#ffddb3", c300="#fdba74", c400="#f29100", c50="#fff7ed", c500="#f97316", c600="#ea580c", c700="#c2410c", c800="#9a3412", c900="#7c2d12", c950="#6c2e12"),
    neutral_hue="slate",
    radius_size="sm",
    font=['VivalaSansRound', 'ui-sans-serif', 'system-ui', 'sans-serif'],
).set(
    embed_radius='*radius_xs',
    border_color_accent='*primary_400',
    border_color_accent_dark='*secondary_700',
    border_color_primary='*secondary_700',
    border_color_primary_dark='*secondary_700',
    color_accent='*primary_400',
    shadow_drop='*shadow_drop_lg',
    button_primary_background_fill='*primary_400',
    button_primary_background_fill_dark='*primary_400',
    button_primary_background_fill_hover='*secondary_700',
    button_primary_background_fill_hover_dark='*secondary_700',
    button_primary_border_color='*secondary_700',
    button_primary_border_color_dark='*secondary_700'
)

# Create a test configuration
test_config = ChatAppConfig(
    app_name="Job Description Assistant",
    description="Chat with an AI to create better job descriptions",
    system_prompt="You are an assistant that helps users create professional job descriptions. Ask questions to gather information about the position and responsibilities.",
    starter_prompt="Hello! I'm your job description assistant. Tell me about the position you'd like to create a description for.",
    model=ModelConfig(
        model_name="mistralai/Mistral-7B-Instruct-v0.3",
        api_key_env_var="HF_API_KEY"
    ),
    theme=themeWDODelta,
    logo_path=Path("../data/wdod_logo.svg")
)

# Create and launch the app
app = create_chat_app(test_config)
app.launch(share=True)  # Set share=False if you don't want a public URL

/tmp/ipykernel_16046/152709889.py:72: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, label="Conversation")


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://686c1d9fdd6f6ddde6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()